# PostgreSQL DB Setup

### Installing Packages: psycopg2-binary

In [2]:
# !pip install psycopg2-binary

    100% |████████████████████████████████| 1.4MB 6.6MB/s 


### Run File with Postgre Credentials

In [1]:
%run sql_test.py

In [2]:
whos

Variable     Type    Data/Info
------------------------------
DBNAME       str     project4
IP_ADDRESS   str     hovies-dsi-project4.cnpp5<...>-west-2.rds.amazonaws.com
PASSWORD     str     general_dsi_p4
USER         str     dsi_project4


### Importing Libraries

In [3]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json
import json, csv
import pandas as pd

### Functions: Connecting to DB | Executing Query

In [4]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS,
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit()
    con.close()

### Importing Data

In [37]:
df = pd.read_csv('./data/hovs_section.csv')

In [38]:
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.head()

,id,name,alias,type,rating,review_count,price,location,latitude,longitude
0,PEHM9AEqq0ca3vACyOMEwA,Lusy's Mediterranean Cafe & Grill,lusys-mediterranean-cafe-and-grill-van-nuys-2,mediterranean,4.5,771,2.0,91401,34.186598,-118.431349
1,ja_cBagHfhI0eFJrw3BRTA,Kobee Factory,kobee-factory-van-nuys-2,mideastern,4.5,536,2.0,91401,34.179265,-118.440370
2,vWuft2V5ZKKWRPzQUHuKDw,Nat's Early Bite Coffee Shop,nats-early-bite-coffee-shop-sherman-oaks,diners,4.5,1069,2.0,91401,34.172400,-118.440530
3,DfmaMh5rJQ_o9vEvhfUDgQ,Uncle Tony's Pizzeria,uncle-tonys-pizzeria-north-hollywood,italian,4.0,1164,2.0,91606,34.187380,-118.416558
4,Mfa5dHJKcY4K-c3IQIxKkA,Krimsey's Cajun Kitchen,krimseys-cajun-kitchen-north-hollywood-2,vegan,4.5,870,2.0,91606,34.186299,-118.413965


### Replacing Commas in 'name' field with periods.

In [39]:
df['name'] = df['name'].map(lambda x: x.replace(',','.'))
df.head()

,id,name,alias,type,rating,review_count,price,location,latitude,longitude
0,PEHM9AEqq0ca3vACyOMEwA,Lusy's Mediterranean Cafe & Grill,lusys-mediterranean-cafe-and-grill-van-nuys-2,mediterranean,4.5,771,2.0,91401,34.186598,-118.431349
1,ja_cBagHfhI0eFJrw3BRTA,Kobee Factory,kobee-factory-van-nuys-2,mideastern,4.5,536,2.0,91401,34.179265,-118.440370
2,vWuft2V5ZKKWRPzQUHuKDw,Nat's Early Bite Coffee Shop,nats-early-bite-coffee-shop-sherman-oaks,diners,4.5,1069,2.0,91401,34.172400,-118.440530
3,DfmaMh5rJQ_o9vEvhfUDgQ,Uncle Tony's Pizzeria,uncle-tonys-pizzeria-north-hollywood,italian,4.0,1164,2.0,91606,34.187380,-118.416558
4,Mfa5dHJKcY4K-c3IQIxKkA,Krimsey's Cajun Kitchen,krimseys-cajun-kitchen-north-hollywood-2,vegan,4.5,870,2.0,91606,34.186299,-118.413965


## Overwriting the old CSV

In [40]:
df.to_csv('./data/hovs_section.csv', index=True)

In [ ]:
# df['latitude'] = df['latitude'].map(lambda x: round(x,6))
# df['longitude'] = df['longitude'].map(lambda x: round(x,6))
# df.head()

## Creating a PostgreSQL table

### Query to Create table

In [36]:
query = ''' CREATE TABLE hovanes_table

(index varchar,
id varchar,
name varchar,
alias varchar,
type varchar,
rating float(2),
review_count int,
price float(2),
location varchar,
latitude float(20),
longitude float(20)
);'''

### Query to Drop Table ~ don't run unless you make a mistake and need to delete table

In [34]:
# query = '''DROP TABLE hovanes_table;'''

### Executing the Query

In [41]:
execute_query(query, command=True)

### Inputting Data Into the Table

In [42]:
con, cur = con_cur_to_db()
with open('./data/hovs_section.csv', 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f)  # Skip the header row.
    cur.copy_from(f, 'hovanes_table', sep=',')

con.commit()

In [12]:
# def insert_entry_json(data, tablename=None):
#     con, cur = con_cur_to_db()
#     for x in data:
#         cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
#     con.commit()
#     con.close()

In [13]:
# insert_entry_json(raw_data, tablename='david_table')

In [12]:
# query = '''SELECT * FROM douglas_table;'''

In [13]:
# response = execute_query(query)

In [ ]:
# response[0]

In [27]:
# query = """SELECT data->'text'
# FROM douglas_table
# WHERE data::text
# LIKE '%RT%';
# """

In [28]:
# response = execute_query(query, dict_cur=True)

In [ ]:
# len(response)

In [30]:
# import pandas as pd

In [ ]:
# pd.DataFrame(response)